In [1]:
import ujson as json
import pandas as pd
import dask.bag as db
from distributed import Client, LocalCluster

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

%matplotlib inline

In [2]:
cluster = LocalCluster(n_workers=8)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:45697 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 16.68 GB


# Sacar Participantes (parties)

In [3]:
def process_row(row):
    parties = row['parties']
    entidad = parties[0]
    unidad = parties[1]
    tender = row['tender']
    general_info = {
        'ocid': row['ocid'],
        'date': row['date'],
        'ID': row.get('id', None),
        'entidad_id': entidad['id'],
        'entidad_nombre': entidad['name'],
        'entidad_role': entidad['roles'][0],
        'unidad_id': unidad['id'],
        'unidad_nombre': unidad['name'],
        'unidad_role': unidad['roles'][0],
        'tender_id': tender['id'],
        'tender_title': tender['title'],
        'tender_description': tender.get('description', None),
        'tender_procurement_method': tender.get('procurementMethod', None),
        'tender_status': tender['status'],
        # quitamos dependencia y unidad
        'n_parties': len(row['parties']) - 2
        
    }
    records = []
    for p in parties[2:]:
        record = {
            **general_info,
            f'integrante_id': p['id'],
            f'integrante_name': p['name'],
        }
        roles = p.get('roles', None)
        if roles:
            roles = sorted(roles)
            roles_str = ','.join(roles)
            record['roles'] = roles_str
        records.append(record)
    return records


# def process_row(row):
#     general_info = get_general_info(row)
#     return general_info


bag = (db.read_text('../../mongo_data/contrataciones.json', blocksize='10MB')
         .map(json.loads)
         .map(process_row)
         .flatten())

results = bag.compute()

df = pd.DataFrame(results)
# No viene un numero
df = df.loc[~df.ocid.isin({'ocds-07smqs-1584896', 'ocds-07smqs-1288029', 'ocds-07smqs-1498712'})]
df = df.assign(
    entidad_id=df.entidad_id.astype('category'),
    tender_procurement_method=df.tender_procurement_method.astype('category'),
    tender_status=df.tender_status.astype('category'),
    # award_status=df.award_status.astype('category'),
    # contract_currency=df.contract_currency.astype('category'),
    # award_currency=df.award_currency.astype('category'),
    date=pd.to_datetime(df.date),
    tender_id=df.tender_id.astype(int),
    # award_id=df.award_id.astype(int),
    # contract_id=df.contract_id.astype(int),
    # award_amount=df.award_amount.astype(float),
    # contract_amount=df.contract_amount.astype(float),
    # contract_period_start=pd.to_datetime(df.contract_period_start),
)

print(df.shape)
df.head()

(590878, 18)


,ocid,date,ID,entidad_id,entidad_nombre,entidad_role,unidad_id,unidad_nombre,unidad_role,tender_id,tender_title,tender_description,tender_procurement_method,tender_status,n_parties,integrante_id,integrante_name,roles
0,ocds-07smqs-1003123,2016-02-19 01:49:22+00:00,SFP-1003123-2018-11-12,PGR-251,Procuraduría General de la República,buyer,PGR850101RC6-017000017,PGR-DELEGACION EN EL DISTRITO FEDERAL #017000017,procuringEntity,1003123,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,direct,complete,3,SCA031118BX7,SERVILLANTAS CONSORCIO AUTOMOTRIZ SA DE CV,tenderer
1,ocds-07smqs-1003123,2016-02-19 01:49:22+00:00,SFP-1003123-2018-11-12,PGR-251,Procuraduría General de la República,buyer,PGR850101RC6-017000017,PGR-DELEGACION EN EL DISTRITO FEDERAL #017000017,procuringEntity,1003123,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,direct,complete,3,SAU0505307M9,SPORTING AUTORREPARACIONES SA DE CV,tenderer
2,ocds-07smqs-1003123,2016-02-19 01:49:22+00:00,SFP-1003123-2018-11-12,PGR-251,Procuraduría General de la República,buyer,PGR850101RC6-017000017,PGR-DELEGACION EN EL DISTRITO FEDERAL #017000017,procuringEntity,1003123,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,direct,complete,3,SCK070618C21,SERVICIOS CORPORATIVOS KEMPER SA DE CV,"supplier,tenderer"
3,ocds-07smqs-1003803,2016-02-19 01:09:18+00:00,SFP-1003803-2018-11-12,SAT-284,Servicio de Administración Tributaria,buyer,SAT970701NN3-006E00002,SAT-Administración de Operación de Recursos y ...,procuringEntity,1003803,SERVICIO INTEGRAL DE CANINOS.,SERVICIO INTEGRAL DE CANINOS.,direct,complete,1,E9C1C827AE1234CCF7AC4D9070BB597C,IRAM LIEVANOS VELAZQUEZ,"supplier,tenderer"
4,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,buyer,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,procuringEntity,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,None,direct,complete,1,R&S811221KR6,SEGUROS SURA SA DE CV,"supplier,tenderer"


In [4]:
df.to_parquet('../data/contrataciones_parties.parquet', engine='pyarrow')
df.to_csv('../data/contrataciones_parties.csv', index=False, quoting=1, encoding='utf-8')

In [6]:
df.roles.value_counts(dropna=False)

supplier,tenderer    419382
tenderer             171496
Name: roles, dtype: int64

In [5]:
# df.loc[df.ocid == 'ocds-07smqs-1252416']

In [6]:
df.unidad_roles.value_counts(dropna=False)

[procuringEntity]    364856
Name: unidad_roles, dtype: int64

In [7]:
df.shape

(590878, 18)

# Sacar participantes (tenders)

In [4]:
def process_row(row):
    parties = row['parties']
    entidad = parties[0]
    unidad = parties[1]
    tender = row['tender']
    tenderers = tender.get('tenderers', None)
    general_info = {
        'ocid': row['ocid'],
        'date': row['date'],
        'ID': row.get('id', None),
        'entidad_id': entidad['id'],
        'entidad_nombre': entidad['name'],
        'unidad_id': unidad['id'],
        'unidad_nombre': unidad['name'],
        'tender_id': tender['id'],
        'tender_title': tender['title'],
        'tender_description': tender.get('tender_description', None),
        'tender_procurement_method': tender.get('procurementMethod', None),
        'tender_status': tender['status'],
        'numberOfTenderers': tender.get('numberOfTenderers', None), 
    }
    if tenderers:
        general_info['len_tenderers'] = len(tenderers)
    return general_info
#     records = []
#     for p in parties[2:]:
#         record = {
#             **general_info,
#             f'integrante_id': p['id'],
#             f'integrante_name': p['name'],
#         }
#         records.append(record)
#     return records



bag = (db.read_text('../../mongo_data/contrataciones.json', blocksize='10MB')
         .map(json.loads)
         .map(process_row))
#          .flatten())

results = bag.compute()

df = pd.DataFrame(results)
# No viene un numero
df = df.loc[~df.ocid.isin({'ocds-07smqs-1584896', 'ocds-07smqs-1288029', 'ocds-07smqs-1498712'})]
df = df.assign(
    entidad_id=df.entidad_id.astype('category'),
    tender_procurement_method=df.tender_procurement_method.astype('category'),
    tender_status=df.tender_status.astype('category'),
    # award_status=df.award_status.astype('category'),
    # contract_currency=df.contract_currency.astype('category'),
    # award_currency=df.award_currency.astype('category'),
    date=pd.to_datetime(df.date),
    tender_id=df.tender_id.astype(int),
    # award_id=df.award_id.astype(int),
    # contract_id=df.contract_id.astype(int),
    # award_amount=df.award_amount.astype(float),
    # contract_amount=df.contract_amount.astype(float),
    # contract_period_start=pd.to_datetime(df.contract_period_start),
)

print(df.shape)
df.head()

(364856, 14)


,ocid,date,ID,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_description,tender_procurement_method,tender_status,numberOfTenderers,len_tenderers
0,ocds-07smqs-1003123,2016-02-19 01:49:22+00:00,SFP-1003123-2018-11-12,PGR-251,Procuraduría General de la República,PGR850101RC6-017000017,PGR-DELEGACION EN EL DISTRITO FEDERAL #017000017,1003123,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,None,direct,complete,3.0,3.0
1,ocds-07smqs-1003803,2016-02-19 01:09:18+00:00,SFP-1003803-2018-11-12,SAT-284,Servicio de Administración Tributaria,SAT970701NN3-006E00002,SAT-Administración de Operación de Recursos y ...,1003803,SERVICIO INTEGRAL DE CANINOS.,None,direct,complete,NaN,NaN
2,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,None,direct,complete,NaN,NaN
3,ocds-07smqs-1012355,2016-03-02 01:58:39+00:00,SFP-1012355-2018-11-12,CONUEE-98,Comisión Nacional para el Uso Eficiente de la ...,CNU800928K31-018E00999,CONUEE-Dirección de Recursos Materiales y Serv...,1012355,SERVICIO INTEGRAL DE ASEGURAMIENTO DE BIENES P...,None,direct,complete,NaN,NaN
4,ocds-07smqs-1017118,2016-03-08 07:17:10+00:00,SFP-1017118-2018-11-12,INAH-195,Instituto Nacional de Antropología e Historia,INA460815GV1-048D00001,INAH-Dir. de la Coordinación de Recursos Mater...,1017118,Servicios de Jardinería en Diversos Inmuebles ...,None,direct,complete,2.0,2.0


In [6]:
df_test = df.loc[~df.numberOfTenderers.isna()]
print(df_test.shape)

(70764, 14)


In [7]:
(df_test.numberOfTenderers == df_test.len_tenderers).value_counts()

True     70574
False      190
dtype: int64

In [13]:
df.loc[df.ocid == 'ocds-07smqs-1252416']

,ocid,date,ID,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_description,tender_procurement_method,tender_status,numberOfTenderers,len_tenderers
75495,ocds-07smqs-1252416,2017-02-09 10:21:20+00:00,SFP-1252416-2018-11-12,CONAFOR-96,Comisión Nacional Forestal,CNF010405EG1-016RHQ001,CONAFOR-Coordinación General de Administración...,1252416,Prestación de servicios profesionales,None,direct,complete,NaN,NaN


In [9]:
# df_test.loc[df_test.numberOfTenderers != df_test.len_tenderers].head(10)

In [10]:
df.loc[df.n_parties == 2].tender_status.value_counts()

active          2299
unsuccessful    2195
complete          55
Name: tender_status, dtype: int64

In [18]:
values = [0, 1, 2, 3]
values = [0, 1]
for v in values[2:]:
    print(v)


In [ ]:
df.loc[df.]

In [1]:
# df.n_parties.value_counts(dropna=False)

# CUCOP

# TODO: ordenar items por ID

In [3]:
def get_general_info(row):
    entidad = row['parties'][0]
    unidad = row['parties'][1]
    tender = row['tender']
    record = {
        'ocid': row['ocid'],
        'date': row['date'],
        'ID': row.get('id', None),
        'entidad_id': entidad['id'],
        'entidad_nombre': entidad['name'],
        'unidad_id': unidad['id'],
        'unidad_nombre': unidad['name'],
        'tender_id': tender['id'],
        'tender_title': tender['title'],
        'tender_description': tender.get('tender_description', None),
        'tender_procurement_method': tender.get('procurementMethod', None),
        'tender_status': tender['status'],
        
    }
    return record

def get_awards_info(awards):
    data = []
    for a in awards:
        award_general_info = {
            'award_id': int(a['id']),
            'award_title': a['title'],
            'award_status': a.get('status', None),
            'award_currency': a['value']['currency'],
            'award_amount': a['value']['amount'],
            'award_supplier_name': a['suppliers'][0]['name'],
            'award_supplier_id': a['suppliers'][0]['id'],
        }
        award_items = a.get('items', None)
        if award_items is None:
            data.append(award_general_info)
            continue
        award_items = sorted(award_items, key=lambda k: int(k['id']))
        for item in award_items:
            # Se repite el award info pero se agregan los items
            # en cada iteracion
            record = {
                **award_general_info,
                'award_item_id': item.get('id', None),
                'award_item_description': item.get('description', None),
                'award_item_quantity': item.get('quantity', None),
            }
            classification = item.get('classification', None)
            unit = item.get('unit', None)
            if classification:
                record['award_classification_scheme'] = classification.get('scheme', None)
                record['award_classification_scheme_id'] = classification.get('id', None)
                record['award_classification_scheme_description'] = classification.get('description', None)
            if unit:
                record['award_unit_name'] = unit['name']
                record['award_unit_value_amount'] = unit['value']['amount']
                record['award_unit_value_currency'] = unit['value']['currency']
            data.append(record)
    return data
            

def get_contracts_info(contracts):
    data = []
    for c in contracts:
        contract_general_info = {
            'contract_id': int(c['id']),
            'contract_title': c['title'],
            'contract_status': c['status'],
            'contract_currency': c['value']['currency'],
            'contract_amount': c['value']['amount'],
            'contract_period_start': c['period']['startDate'],
        }
        contract_items = c.get('items', None)
        if contract_items is None:
            data.append(contract_general_info)
            continue
        contract_items = sorted(contract_items, key=lambda k: int(k['id']))
        for item in contract_items:
            # Se repite el contract info pero se agregan los items
            # en cada iteracion
            record = {
                **contract_general_info,
                'contract_item_id': item.get('id', None),
                'contract_item_description': item.get('description', None),
                'contract_item_quantity': item.get('quantity', None),
            }
            classification = item.get('classification', None)
            unit = item.get('unit', None)
            if classification:
                record['contract_classification_scheme'] = classification.get('scheme', None)
                record['contract_classification_scheme_id'] = classification.get('id', None)
                record['contract_classification_scheme_description'] = classification.get('description', None)
            if unit:
                record['contract_unit_name'] = unit['name']
                record['contract_unit_value_amount'] = unit['value']['amount']
                record['contract_unit_value_currency'] = unit['value']['currency']
            data.append(record)
    return data
            

# TODO: revisar si podemos combinar los dicts usando Bag
def process_row(row):
    general_info = get_general_info(row)
    awards = row.get('awards', None)
    contracts = row.get('contracts', None)
    # solo si existen contracts y awards vamos a regresar info
    if awards and contracts:
        # ordenamos para asegurar que sean iguales
        awards = sorted(awards, key=lambda k: int(k['id']))
        contracts = sorted(contracts, key=lambda k: int(k['awardID']))
        awards_info = get_awards_info(awards)
        contracts_info = get_contracts_info(contracts)
        records = []
        for a_info, c_info in zip(awards_info, contracts_info):
            record = {
                **general_info,
                **a_info, 
                **c_info
            }
            records.append(record)
        return records
    return None


# award_unit_value_amount, contract_unit_value_amount

def remove_dicts(row):
    award_unit_value_amount = row.get('award_unit_value_amount')
    contract_unit_value_amount = row.get('contract_unit_value_amount')
    if isinstance(award_unit_value_amount, dict):
        row['award_unit_value_amount'] = None
    if isinstance(contract_unit_value_amount, dict):
        row['contract_unit_value_amount'] = None  
    return row


bag = (db.read_text('../../mongo_data/contrataciones.json', blocksize='10MB')
         .map(json.loads)
         .map(process_row)
         .filter(lambda x: x is not None)
         .flatten()
         .map(remove_dicts))

results = bag.compute()

df = pd.DataFrame(results)
# No viene un numero
df = df.loc[~df.ocid.isin({'ocds-07smqs-1584896', 'ocds-07smqs-1288029', 'ocds-07smqs-1498712'})]
df = df.assign(
    entidad_id=df.entidad_id.astype('category'),
    tender_procurement_method=df.tender_procurement_method.astype('category'),
    tender_status=df.tender_status.astype('category'),
    award_status=df.award_status.astype('category'),
    contract_currency=df.contract_currency.astype('category'),
    award_currency=df.award_currency.astype('category'),
    date=pd.to_datetime(df.date),
    tender_id=df.tender_id.astype(int),
    award_id=df.award_id.astype(int),
    contract_id=df.contract_id.astype(int),
    award_amount=df.award_amount.astype(float),
    contract_amount=df.contract_amount.astype(float),
    contract_period_start=pd.to_datetime(df.contract_period_start),
)

print(df.shape)
df.head()

(1092747, 43)


,ocid,date,ID,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_description,tender_procurement_method,tender_status,award_id,award_title,award_status,award_currency,award_amount,award_supplier_name,award_supplier_id,contract_id,contract_title,contract_status,contract_currency,contract_amount,contract_period_start,award_item_id,award_item_description,award_item_quantity,award_classification_scheme,award_classification_scheme_id,award_classification_scheme_description,award_unit_name,award_unit_value_amount,award_unit_value_currency,contract_item_id,contract_item_description,contract_item_quantity,contract_classification_scheme,contract_classification_scheme_id,contract_classification_scheme_description,contract_unit_name,contract_unit_value_amount,contract_unit_value_currency
0,ocds-07smqs-1003123,2016-02-19 01:49:22+00:00,SFP-1003123-2018-11-12,PGR-251,Procuraduría General de la República,PGR850101RC6-017000017,PGR-DELEGACION EN EL DISTRITO FEDERAL #017000017,1003123,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,None,direct,complete,1377619,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,active,MXN,168000.00,SERVICIOS CORPORATIVOS KEMPER SA DE CV,SCK070618C21,1377619,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,terminated,MXN,168000.00,2017-01-01 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ocds-07smqs-1003803,2016-02-19 01:09:18+00:00,SFP-1003803-2018-11-12,SAT-284,Servicio de Administración Tributaria,SAT970701NN3-006E00002,SAT-Administración de Operación de Recursos y ...,1003803,SERVICIO INTEGRAL DE CANINOS.,None,direct,complete,1004248,SERVICIO INTEGRAL DE CANINOS.,active,MXN,8451072.00,IRAM LIEVANOS VELAZQUEZ,E9C1C827AE1234CCF7AC4D9070BB597C,1004248,SERVICIO INTEGRAL DE CANINOS.,terminated,MXN,8451072.00,2017-01-01 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,None,direct,complete,1011086,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES,active,MXN,420689.55,SEGUROS SURA SA DE CV,R&S811221KR6,1011086,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES,terminated,MXN,420689.55,2016-03-01 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,None,direct,complete,1322882,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,active,MXN,50482.75,SEGUROS SURA SA DE CV,R&S811221KR6,1322882,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,terminated,MXN,50482.75,2016-03-01 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,None,direct,complete,1357879,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,active,MXN,33655.17,SEGUROS SURA SA DE CV,R&S811221KR6,1357879,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,terminated,MXN,33655.17,2017-02-28 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.to_csv('../data/contrataciones_items.csv', index=False, quoting=1, encoding='utf-8', sep='>')

In [11]:
!wc -l  ../data/contrataciones_items.csv

1427402 ../data/contrataciones_items.csv


In [9]:
df.shape

(1092747, 43)

In [8]:
df.to_parquet('../data/contrataciones_items.parquet', engine='pyarrow')


In [4]:
df_test = pd.read_parquet('../data/contrataciones_items.parquet', engine='pyarrow')
print(df_test.shape)
df_test.head()

(1092747, 43)


,ocid,date,ID,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_description,tender_procurement_method,tender_status,award_id,award_title,award_status,award_currency,award_amount,award_supplier_name,award_supplier_id,contract_id,contract_title,contract_status,contract_currency,contract_amount,contract_period_start,award_item_id,award_item_description,award_item_quantity,award_classification_scheme,award_classification_scheme_id,award_classification_scheme_description,award_unit_name,award_unit_value_amount,award_unit_value_currency,contract_item_id,contract_item_description,contract_item_quantity,contract_classification_scheme,contract_classification_scheme_id,contract_classification_scheme_description,contract_unit_name,contract_unit_value_amount,contract_unit_value_currency
0,ocds-07smqs-1003123,2016-02-19 01:49:22+00:00,SFP-1003123-2018-11-12,PGR-251,Procuraduría General de la República,PGR850101RC6-017000017,PGR-DELEGACION EN EL DISTRITO FEDERAL #017000017,1003123,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,None,direct,complete,1377619,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,active,MXN,168000.00,SERVICIOS CORPORATIVOS KEMPER SA DE CV,SCK070618C21,1377619,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,terminated,MXN,168000.00,2017-01-01 12:00:00+00:00,None,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,None,NaN,None
1,ocds-07smqs-1003803,2016-02-19 01:09:18+00:00,SFP-1003803-2018-11-12,SAT-284,Servicio de Administración Tributaria,SAT970701NN3-006E00002,SAT-Administración de Operación de Recursos y ...,1003803,SERVICIO INTEGRAL DE CANINOS.,None,direct,complete,1004248,SERVICIO INTEGRAL DE CANINOS.,active,MXN,8451072.00,IRAM LIEVANOS VELAZQUEZ,E9C1C827AE1234CCF7AC4D9070BB597C,1004248,SERVICIO INTEGRAL DE CANINOS.,terminated,MXN,8451072.00,2017-01-01 12:00:00+00:00,None,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,None,NaN,None
2,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,None,direct,complete,1011086,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES,active,MXN,420689.55,SEGUROS SURA SA DE CV,R&S811221KR6,1011086,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES,terminated,MXN,420689.55,2016-03-01 12:00:00+00:00,None,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,None,NaN,None
3,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,None,direct,complete,1322882,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,active,MXN,50482.75,SEGUROS SURA SA DE CV,R&S811221KR6,1322882,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,terminated,MXN,50482.75,2016-03-01 12:00:00+00:00,None,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,None,NaN,None
4,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,None,direct,complete,1357879,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,active,MXN,33655.17,SEGUROS SURA SA DE CV,R&S811221KR6,1357879,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,terminated,MXN,33655.17,2017-02-28 12:00:00+00:00,None,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,None,NaN,None


In [4]:
(df.award_item_id == df.contract_item_id).value_counts()

True     802116
False    290631
dtype: int64

In [7]:
df_test = df.loc[(~df.award_item_id.isna()) | (~df.contract_item_id.isna())]
(df_test.award_item_id == df_test.contract_item_id).value_counts()

True    802116
dtype: int64

In [3]:
import csv


In [5]:
df_test.to_csv('../data/contrataciones_items.csv', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8', sep='|')

In [6]:
!wc -l  ../data/contrataciones_items.csv

1427402 ../data/contrataciones_items.csv


In [21]:
for r in df.itertuples():
    try:
        float(r.award_unit_value_amount)
    except TypeError:
        print(r.ocid)
        break

ocds-07smqs-1610425


In [27]:
r.award_unit_value_amount

{'$numberLong': '7520996067'}

In [2]:
# df.loc[125754]

In [22]:
df_test = df.loc[df.ocid == 'ocds-07smqs-1610425']
print(df_test.shape)

(8, 43)


In [5]:
df_test = df_test.loc[(~df_test.award_item_id.isna()) | (~df_test.contract_item_id.isna())]


In [10]:
df_aux = df_test.loc[df_test.award_item_id != df_test.contract_item_id]
df_aux.tail()

,ocid,date,ID,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_description,tender_procurement_method,tender_status,award_id,award_title,award_status,award_currency,award_amount,award_supplier_name,award_supplier_id,contract_id,contract_title,contract_status,contract_currency,contract_amount,contract_period_start,award_item_id,award_item_description,award_item_quantity,award_classification_scheme,award_classification_scheme_id,award_classification_scheme_description,award_unit_name,award_unit_value_amount,award_unit_value_currency,contract_item_id,contract_item_description,contract_item_quantity,contract_classification_scheme,contract_classification_scheme_id,contract_classification_scheme_description,contract_unit_name,contract_unit_value_amount,contract_unit_value_currency
1083718,ocds-07smqs-1877006,2019-03-14 12:40:27+00:00,SFP-1877006-2019-06-12,IMSS-192,Instituto Mexicano del Seguro Social,IMS421231I45-050GYR067,IMSS-UMAE HOSPITAL DE PEDIATRÍA SILVESTRE FREN...,1877006,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",None,direct,complete,2022871,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",active,MXN,4267405.0,Impromed,IMP920626FVA,2022871,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",active,MXN,4267405.0,2019-03-01 12:00:00+00:00,5749386,Glucosa,1.0,CUCOP: Clasificador Único de las Contratacione...,33900008,Servicio de estudios clinicos,Servicio,7.78,MXN,5749387,HDL Colesterol,1.0,NaN,33900008,Servicio de estudios clinicos,Servicio,7.73,MXN
1083750,ocds-07smqs-1877006,2019-03-14 12:40:27+00:00,SFP-1877006-2019-06-12,IMSS-192,Instituto Mexicano del Seguro Social,IMS421231I45-050GYR067,IMSS-UMAE HOSPITAL DE PEDIATRÍA SILVESTRE FREN...,1877006,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",None,direct,complete,2022871,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",active,MXN,4267405.0,Impromed,IMP920626FVA,2022871,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",active,MXN,4267405.0,2019-03-01 12:00:00+00:00,5749394,Proteínas Totales,1.0,CUCOP: Clasificador Único de las Contratacione...,33900008,Servicio de estudios clinicos,Servicio,7.78,MXN,5749391,Magnesio,1.0,NaN,33900008,Servicio de estudios clinicos,Servicio,7.76,MXN
1083751,ocds-07smqs-1877006,2019-03-14 12:40:27+00:00,SFP-1877006-2019-06-12,IMSS-192,Instituto Mexicano del Seguro Social,IMS421231I45-050GYR067,IMSS-UMAE HOSPITAL DE PEDIATRÍA SILVESTRE FREN...,1877006,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",None,direct,complete,2022871,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",active,MXN,4267405.0,Impromed,IMP920626FVA,2022871,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",active,MXN,4267405.0,2019-03-01 12:00:00+00:00,5749391,Magnesio,1.0,CUCOP: Clasificador Único de las Contratacione...,33900008,Servicio de estudios clinicos,Servicio,7.76,MXN,5749392,Microalbúmina,1.0,NaN,33900008,Servicio de estudios clinicos,Servicio,7.72,MXN
1083752,ocds-07smqs-1877006,2019-03-14 12:40:27+00:00,SFP-1877006-2019-06-12,IMSS-192,Instituto Mexicano del Seguro Social,IMS421231I45-050GYR067,IMSS-UMAE HOSPITAL DE PEDIATRÍA SILVESTRE FREN...,1877006,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",None,direct,complete,2022871,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",active,MXN,4267405.0,Impromed,IMP920626FVA,2022871,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",active,MXN,4267405.0,2019-03-01 12:00:00+00:00,5749392,Microalbúmina,1.0,CUCOP: Clasificador Único de las Contratacione...,33900008,Servicio de estudios clinicos,Servicio,7.72,MXN,5749393,Microproteínas,1.0,NaN,33900008,Servicio de estudios clinicos,Servicio,7.80,MXN
1083753,ocds-07smqs-1877006,2019-03-14 12:40:27+00:00,SFP-1877006-2019-06-12,IMSS-192,Instituto Mexicano del Seguro Social,IMS421231I45-050GYR067,IMSS-UMAE HOSPITAL DE PEDIATRÍA SILVESTRE FREN...,1877006,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",None,direct,complete,2022871,"SERVICIO INTEGRAL DE LABORATORIO PAQUETES 1,2,...",active,MXN,4267405.0,Impromed

In [3]:
# df_test.loc[df_test.ocid == 'ocds-07smqs-1877006', ['award_item_id', 'award_item_description', 'contract_item_id', 'contract_item_description']].sort_values('award_item_id')

In [4]:
# df_test.loc[df_test.ocid == 'ocds-07smqs-1877006'].sort_values('award_item_id').drop('award_classification_scheme', axis=1)

In [6]:
df_test = df.loc[(~df.award_item_id.isna()) | (~df.contract_item_id.isna())]
print(df_test.shape)


(802116, 43)


In [7]:
df_test.ocid.nunique()

99135

In [9]:
(df_test.award_item_id == df_test.contract_item_id).value_counts()

True     749860
False     52256
dtype: int64

In [5]:
# df_test.award_classification_scheme_description.value_counts(dropna=False)

In [11]:
df_aux = df_test.groupby(['award_classification_scheme_description', 'award_unit_name'], as_index=False).award_classification_scheme_id.count()
df_aux.sort_values('award_classification_scheme_id', ascending=False)

,award_classification_scheme_description,award_unit_name,award_classification_scheme_id
24234,Servicios integrales,Servicio,17557
13683,Estudios e investigaciones,Servicio,16985
17790,Maquinaria y equipo (mantenimiento y reparacion),Servicio,10424
4265,Accesorios de equipo e instrumental medico y d...,Pieza,9897
24043,Servicios de mantenimiento de equipo e instrum...,Servicio,8158
...,...,...,...
14173,Flores de todas clases (arboles o plantas),Rollo,1
14172,Flores de todas clases (arboles o plantas),Manojo,1
3377,010.000.5694.00 Somatropina,Envase,1
3375,010.000.5691.00 Desmopresina,Caja,1


# Datos ganadores

# En caso de que no haya dato de items qué hacemos? sólo dejo un registro?

In [3]:
def process_row(row):    
    awards = row.get('awards', None)
    contracts = row.get('contracts', None)
    # solo si existen contracts y awards vamos a regresar info
    if awards and contracts:
        entidad = row['parties'][0]
        unidad = row['parties'][1]
        tender = row['tender']
        # ordenamos para asegurar que sean iguales
        awards = sorted(awards, key=lambda k: int(k['id']))
        contracts = sorted(contracts, key=lambda k: int(k['awardID']))
        # Por cada caso vamos a repetir la info
        records = []
        for a, c in zip(awards, contracts):
            award_items = a.get('items', None)
            contract_items = c.get('items', None)
            if award_items and contract_items:
                # n_award_items = len(award_items)
                # n_contract_items = len(contract_items)
            for a
            record = {
                # info rep
                'ocid': row['ocid'],
                'date': row['date'],
                'ID': row.get('id', None),
                'entidad_id': entidad['id'],
                'entidad_nombre': entidad['name'],
                'unidad_id': unidad['id'],
                'unidad_nombre': unidad['name'],
                'tender_id': tender['id'],
                'tender_title': tender['title'],
                'tender_description': tender.get('tender_description', None),
                'tender_procurement_method': tender.get('procurementMethod', None),
                'tender_status': tender['status'],
                # info no rep
                'award_id': a['id'],
                'award_title': a['title'],
                'award_status': a.get('status', None),
                'award_currency': a['value']['currency'],
                'award_amount': a['value']['amount'],
                'award_supplier_name': a['suppliers'][0]['name'],
                'award_supplier_id': a['suppliers'][0]['id'],
                # 'award_items': n_award_items,
                'contract_id': c['id'],
                'contract_title': c['title'],
                'contract_status': c['status'],
                # 'contract_items': n_contract_items,
                'contract_currency': c['value']['currency'],
                'contract_amount': c['value']['amount'],
                'contract_period_start': c['period']['startDate'],
            }
            records.append(record)
        return records
    return None


bag = (db.read_text('../../mongo_data/contrataciones.json', blocksize='30MB')
         .map(json.loads)
         .map(process_row)
         .filter(lambda x: x is not None)
         .flatten())

results = bag.compute()

df = pd.DataFrame(results)
# No viene un numero
df = df.loc[~df.ocid.isin({'ocds-07smqs-1584896', 'ocds-07smqs-1288029', 'ocds-07smqs-1498712'})]
df = df.assign(
    date=pd.to_datetime(df.date),
    tender_id=df.tender_id.astype(int),
    award_id=df.award_id.astype(int),
    contract_id=df.contract_id.astype(int),
    award_amount=df.award_amount.astype(float),
    contract_amount=df.contract_amount.astype(float),
    contract_period_start=pd.to_datetime(df.contract_period_start),
    tender_procurement_method=df.tender_procurement_method.astype('category'),
    tender_status=df.tender_status.astype('category'),
    award_status=df.award_status.astype('category'),
)

print(df.shape)
df.head()

(446512, 27)


,ocid,date,ID,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_procurement_method,tender_description,tender_status,award_id,award_title,award_status,award_currency,award_amount,award_supplier_name,award_supplier_id,award_items,contract_id,contract_title,contract_status,contract_items,contract_currency,contract_amount,contract_period_start
0,ocds-07smqs-1003123,2016-02-19 01:49:22+00:00,SFP-1003123-2018-11-12,PGR-251,Procuraduría General de la República,PGR850101RC6-017000017,PGR-DELEGACION EN EL DISTRITO FEDERAL #017000017,1003123,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,direct,None,complete,1377619,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,active,MXN,168000.00,SERVICIOS CORPORATIVOS KEMPER SA DE CV,SCK070618C21,NaN,1377619,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,terminated,NaN,MXN,168000.00,2017-01-01 12:00:00+00:00
1,ocds-07smqs-1003803,2016-02-19 01:09:18+00:00,SFP-1003803-2018-11-12,SAT-284,Servicio de Administración Tributaria,SAT970701NN3-006E00002,SAT-Administración de Operación de Recursos y ...,1003803,SERVICIO INTEGRAL DE CANINOS.,direct,None,complete,1004248,SERVICIO INTEGRAL DE CANINOS.,active,MXN,8451072.00,IRAM LIEVANOS VELAZQUEZ,E9C1C827AE1234CCF7AC4D9070BB597C,NaN,1004248,SERVICIO INTEGRAL DE CANINOS.,terminated,NaN,MXN,8451072.00,2017-01-01 12:00:00+00:00
2,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,direct,None,complete,1011086,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES,active,MXN,420689.55,SEGUROS SURA SA DE CV,R&S811221KR6,NaN,1011086,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES,terminated,NaN,MXN,420689.55,2016-03-01 12:00:00+00:00
3,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,direct,None,complete,1322882,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,active,MXN,50482.75,SEGUROS SURA SA DE CV,R&S811221KR6,NaN,1322882,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,terminated,NaN,MXN,50482.75,2016-03-01 12:00:00+00:00
4,ocds-07smqs-1009245,2016-02-26 05:33:08+00:00,SFP-1009245-2018-11-12,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,direct,None,complete,1357879,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,active,MXN,33655.17,SEGUROS SURA SA DE CV,R&S811221KR6,NaN,1357879,PÓLIZA DE SEGURO DE ACCIDENTES PERSONALES (MOD...,terminated,NaN,MXN,33655.17,2017-02-28 12:00:00+00:00


In [8]:
(df.contract_items == df.award_items).value_counts(dropna=False)

False    290631
True     155881
dtype: int64

In [14]:
df_test = df.loc[~df.contract_items.isna()]
(df_test.contract_items == df_test.award_items).value_counts(dropna=False)

True    155881
dtype: int64

In [1]:
# df.loc[df.contract_items == 40]

In [5]:
df.award_items.value_counts(dropna=False)

In [4]:
df.to_parquet('../data/contrataciones_network.parquet', engine='pyarrow')

In [4]:
df.tender_procurement_method.value_counts(dropna=False)

direct       367224
open          51010
selective     28259
NaN              19
Name: tender_procurement_method, dtype: int64

In [9]:
df.award_supplier_id.nunique(), df.award_supplier_name.nunique()

(81658, 86139)

In [7]:
(df.award_id == df.contract_id).value_counts()

True    446512
dtype: int64

In [30]:
datos[196499]['contracts']

[{'id': 1871571,
  'awardID': '1871571',
  'title': 'CONTRATACION DE PRESTADOR DE SERVICIOS PROFESIONALES EN OFICINAS CENTRALES',
  'description': 'CONTRATACION PRESTADOR DE SERVICIOS PROFESIONALES COMO APOYO AL PROGRAMA PRODEZA 2018 EN OFICINAS CENTRALES DE LA COMISION NACIONAL DE LAS ZONAS ARIDAS',
  'status': 'active',
  'period': {'startDate': '2018-07-23T09:00:00Z',
   'endDate': '2018-11-30T11:59:00Z'},
  'value': {'amount': 63763.1, 'currency': 'MXN'},
  'items': [{'id': '5466113',
    'description': 'CONTRATACION PRESTADOR DE SERVICIOS PROFESIONALES COMO APOYO AL PROGRAMA PRODEZA 2018 EN OFICINAS CENTRALES DE LA COMISION NACIONAL DE LAS ZONAS ARIDAS',
    'classification': {'id': '33100002',
     'description': 'Asesorias para la operacion de programas'},
    'quantity': 1,
    'unit': {'name': 'Servicio',
     'value': {'amount': 63793.1, 'currency': 'MXN'}}}],
  'dateSigned': '2018-07-23T00:00:00Z'}]

# Tabla montos

In [3]:

def process_row(row):
    entidad = row['parties'][0]
    unidad = row['parties'][1]
    tender = row['tender']
    result = {
        'ocid': row['ocid'],
        'entidad_id': entidad['id'],
        'entidad_nombre': entidad['name'],
        'unidad_id': unidad['id'],
        'unidad_nombre': unidad['name'],
        'tender_id': tender['id'],
        'tender_title': tender['title'],
        'tender_status': tender['status'],
        # Opcionales
        'tender_submission_method': tender['submissionMethod'][0],
        'tender_procurement_method': tender.get('procurementMethod', None),
        'tender_description': tender.get('tender_description', None),
    }
    
    awards = row.get('awards', None)
    if awards is not None:
        result['n_awards'] = len(awards)
        # result['awards_amount'] = [a['value']['amount'] for a in awards]
        if row['ocid'] not in {'ocds-07smqs-1584896', 'ocds-07smqs-1288029', 'ocds-07smqs-1498712'}:
            result['awards_amount'] = sum(a['value']['amount'] for a in awards)
        result['awards_currencies'] = ','.join(sorted({a['value']['currency'] for a in awards}))
    contracts = row.get('contracts', None)
    if contracts is not None:
        result['n_contracts'] = len(contracts)
        if row['ocid'] not in {'ocds-07smqs-1584896', 'ocds-07smqs-1288029', 'ocds-07smqs-1498712'}:
            result['contracts_amount'] = sum(c['value']['amount'] for c in contracts)
        result['contracts_currencies'] = ','.join(sorted({c['value']['currency'] for c in contracts}))
    return result


# 'buyer_id': row['buyer']['id'],
bag = (db.read_text('../../mongo_data/contrataciones.json', blocksize='30MB')
         .map(json.loads).map(process_row))
results = bag.compute()

# TODO:


df = pd.DataFrame(results)
df = df.assign(tender_id=df.tender_id.astype(int))

print(df.shape)
df.head()

(364860, 17)


,ocid,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_status,tender_submission_method,tender_procurement_method,tender_description,n_awards,awards_amount,awards_currencies,n_contracts,contracts_amount,contracts_currencies
0,ocds-07smqs-1003123,PGR-251,Procuraduría General de la República,PGR850101RC6-017000017,PGR-DELEGACION EN EL DISTRITO FEDERAL #017000017,1003123,MANTENIMIENTO PREVENTIVO Y CORRECTIVO AL PARQU...,complete,electronicSubmission,direct,None,1.0,168000.00,MXN,1.0,168000.00,MXN
1,ocds-07smqs-1003803,SAT-284,Servicio de Administración Tributaria,SAT970701NN3-006E00002,SAT-Administración de Operación de Recursos y ...,1003803,SERVICIO INTEGRAL DE CANINOS.,complete,inPerson,direct,None,1.0,8451072.00,MXN,1.0,8451072.00,MXN
2,ocds-07smqs-1009245,STPS-278,Secretaría del Trabajo y Previsión Social,STP401231P53-014000999,STPS-Dirección de Adquisiciones y Almacenes #0...,1009245,CONTRATACION DE UNA POLIZA DE SEGUROS PERSONALES,complete,written,direct,None,3.0,504827.47,MXN,3.0,504827.47,MXN
3,ocds-07smqs-1012355,CONUEE-98,Comisión Nacional para el Uso Eficiente de la ...,CNU800928K31-018E00999,CONUEE-Dirección de Recursos Materiales y Serv...,1012355,SERVICIO INTEGRAL DE ASEGURAMIENTO DE BIENES P...,complete,written,direct,None,3.0,121173.79,MXN,3.0,121173.79,MXN
4,ocds-07smqs-1017118,INAH-195,Instituto Nacional de Antropología e Historia,INA460815GV1-048D00001,INAH-Dir. de la Coordinación de Recursos Mater...,1017118,Servicios de Jardinería en Diversos Inmuebles ...,complete,electronicSubmission,direct,None,3.0,13505684.89,MXN,3.0,13505684.89,MXN


In [6]:
# # busca ocid
# with open('../../mongo_data/contrataciones.json') as file:
#     datos = {}
#     for i, line in enumerate(file):
#         row = json.loads(line)
#         if row['ocid'] in {'ocds-07smqs-1755349'}:
#             datos[i] = row
        
# print(json.dumps(datos, sort_keys=False, indent=2))



In [18]:
datos[196499]['date'] == datos[320795]['date']

True

In [14]:
datos[196499]['contracts']

[{'id': 1871571,
  'awardID': '1871571',
  'title': 'CONTRATACION DE PRESTADOR DE SERVICIOS PROFESIONALES EN OFICINAS CENTRALES',
  'description': 'CONTRATACION PRESTADOR DE SERVICIOS PROFESIONALES COMO APOYO AL PROGRAMA PRODEZA 2018 EN OFICINAS CENTRALES DE LA COMISION NACIONAL DE LAS ZONAS ARIDAS',
  'status': 'active',
  'period': {'startDate': '2018-07-23T09:00:00Z',
   'endDate': '2018-11-30T11:59:00Z'},
  'value': {'amount': 63763.1, 'currency': 'MXN'},
  'items': [{'id': '5466113',
    'description': 'CONTRATACION PRESTADOR DE SERVICIOS PROFESIONALES COMO APOYO AL PROGRAMA PRODEZA 2018 EN OFICINAS CENTRALES DE LA COMISION NACIONAL DE LAS ZONAS ARIDAS',
    'classification': {'id': '33100002',
     'description': 'Asesorias para la operacion de programas'},
    'quantity': 1,
    'unit': {'name': 'Servicio',
     'value': {'amount': 63793.1, 'currency': 'MXN'}}}],
  'dateSigned': '2018-07-23T00:00:00Z'}]

In [4]:
df.contracts_currencies.value_counts(dropna=False)

MXN            348221
NaN             13075
USD              3057
EUR               288
MXN,USD           123
TEST               37
GBP                28
CAD                22
EUR,MXN,USD         3
EUR,MXN             3
MXN,TEST            1
JPY                 1
EUR,USD             1
Name: contracts_currencies, dtype: int64

In [14]:
# 30898: [{'$numberLong': '2474999688'}]  
for r in df.itertuples():
    if r.Index in {30898,279502, 352981} or isinstance(r.awards_amount, float):
        continue
    sum(r.awards_amount)

In [5]:
sorted({10, 2, 30})

[2, 10, 30]

In [6]:
df.loc[df.ocid == 'ocds-07smqs-1584896']

,ocid,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_status,tender_submission_method,tender_procurement_method,tender_description,n_awards,awards_amounts,awards_currencies,n_contracts,contracts_amount,contracts_currencies
30898,ocds-07smqs-1584896,SAT-284,Servicio de Administración Tributaria,SAT970701NN3-006E00001,SAT-Administración General de Recursos y Servi...,1584896,Servicio de Dictamen de Imágenes de Revisión N...,complete,inPerson,direct,None,1.0,[{'$numberLong': '2474999688'}],1.0,1.0,[{'$numberLong': '2474999688'}],1.0


In [15]:
df.loc[df.awards_currencies == 3]

,ocid,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_status,tender_submission_method,tender_procurement_method,tender_description,n_awards,awards_currencies,n_contracts,contracts_currencies
27958,ocds-07smqs-1545788,SEP-265,Secretaría de Educación Pública,SEP210905778-011000999,SEP-Dirección de Adquisiciones #011000999,1545788,"CONTRATACION DEL SERVICIO PARA LA ""LICENCIA DE...",complete,[electronicSubmission],direct,None,43.0,3.0,43.0,3.0
178791,ocds-07smqs-1681534,CIO-52,"Centro de Investigaciones en Óptica, A.C.",CIO8004181K5-03890S999,CIO-Departamento de Servicios Generales #03890...,1681534,AD CON PEDIDO ENERO - ABRIL 2018 - CAP. 2000,complete,[electronicSubmission],direct,None,12.0,3.0,12.0,3.0
202904,ocds-07smqs-1773210,CIO-52,"Centro de Investigaciones en Óptica, A.C.",CIO8004181K5-03890S999,CIO-Departamento de Servicios Generales #03890...,1773210,AD CON PEDIDO CAP. 3000 JULIO 2018,complete,[electronicSubmission],direct,None,6.0,3.0,6.0,3.0


In [9]:
row['awards'][0]['value']['amount']

20621.91

In [5]:
# df[df.entidad_id == 'IMSS-192']

In [9]:
df.tender_status.value_counts(dropna=False)

complete        351467
unsuccessful      7093
active            6300
Name: tender_status, dtype: int64

In [9]:
df[df.awards.isna()].tender_status.value_counts(dropna=False)

unsuccessful    7050
active          5409
complete         616
Name: tender_status, dtype: int64

In [6]:
# df.n_contracts.value_counts(dropna=False)

In [7]:
# df.n_awards.value_counts(dropna=False)

In [5]:
df.tender_procurement_method.value_counts(dropna=False)

direct       308090
open          30406
selective     26347
NaN              17
Name: tender_procurement_method, dtype: int64

In [4]:
# df[df.tender_procurement_method.isna()]

In [7]:
df.tender_status.value_counts()

complete        351467
unsuccessful      7093
active            6300
Name: tender_status, dtype: int64

In [10]:
df[df.ocid == 'ocds-07smqs-1351177']

,ocid,entidad_id,entidad_nombre,unidad_id,unidad_nombre,tender_id,tender_title,tender_status,tender_procurement_method,awards
12745,ocds-07smqs-1351177,CNBV-80,Comisión Nacional Bancaria y de Valores,CNB950501PT6-006B00001,CNBV-Dirección General Adjunta de Adquisicione...,1351177,Servicios Medicos en la especialidad en Médic...,complete,direct,1.0
292527,ocds-07smqs-1351177,CNBV-80,Comisión Nacional Bancaria y de Valores,CNB950501PT6-006B00001,CNBV-Dirección General Adjunta de Adquisicione...,1351177,Servicios Medicos en la especialidad en Médic...,complete,direct,1.0


In [16]:
df.tender_id.astype(int).sum()

581887352716

In [6]:
# (df.buyer_id == df.entidad_id).value_counts()  # True    364860

In [ ]:
# ocds-07smqs-1351177

In [8]:
df.ocid.value_counts()

ocds-07smqs-1351177    2
ocds-07smqs-1601684    2
ocds-07smqs-1539007    2
ocds-07smqs-1605222    2
ocds-07smqs-1741776    2
                      ..
ocds-07smqs-1434741    1
ocds-07smqs-1797848    1
ocds-07smqs-1452451    1
ocds-07smqs-1885039    1
ocds-07smqs-1850878    1
Name: ocid, Length: 348027, dtype: int64

In [12]:
# pd.DataFrame(
#     result, columns=['year', 'conteo']
# ).sort_values('year').set_index('year').plot.bar()

In [7]:
# with open('../../mongo_data/contrataciones.json') as file:
#     for i, line in enumerate(file):
#         if i == 202904:
#             row = json.loads(line)
#             break
# print(json.dumps(row, sort_keys=False, indent=2))

In [3]:
df_buyers = pd.read_json('../../datosDataton2019/S6_Contrataciones/buyers.json')
print(df_buyers.shape)
df_buyers.head()

(268, 2)


,name,id
0,Procuraduría General de la República,PGR-251
1,Servicio de Administración Tributaria,SAT-284
2,Secretaría del Trabajo y Previsión Social,STPS-278
3,Comisión Nacional para el Uso Eficiente de la ...,CONUEE-98
4,Instituto Nacional de Antropología e Historia,INAH-195


In [19]:
df_tenderers = pd.read_json('../../datosDataton2019/S6_Contrataciones/tenderers_suppliers.json')
df_tenderers = df_tenderers.assign(name=df_tenderers.name.str.upper())
print(df_tenderers.shape)
df_tenderers.head()

(95016, 7)


,_id,name,id,identifier,address,contactPoint,roles
0,{'$oid': '5dd72af19e09317019934a6e'},SERVILLANTAS CONSORCIO AUTOMOTRIZ SA DE CV,SCA031118BX7,"{'id': 'SCA031118BX7', 'legalName': 'SERVILLAN...","{'streetAddress': 'JOSE MORAN 66', 'locality':...","{'email': 'servillantas@prodigy.net.mx', 'tele...",[tenderer]
1,{'$oid': '5dd72af19e09317019934a6f'},SPORTING AUTORREPARACIONES SA DE CV,SAU0505307M9,"{'id': 'SAU0505307M9', 'legalName': 'SPORTING ...",{'streetAddress': 'PLUTARCO ELIAS CALLES No. 6...,{'email': 'sportingautoreparaciones@gmail.com'...,[tenderer]
2,{'$oid': '5dd72af19e09317019934a70'},SERVICIOS CORPORATIVOS KEMPER SA DE CV,SCK070618C21,"{'id': 'SCK070618C21', 'legalName': 'SERVICIOS...",{'streetAddress': 'CALZADA VALLEJO NUMERO 1020...,"{'name': 'EDGAR GUSTAVO TREJO KEMPER', 'email'...","[tenderer, supplier]"
3,{'$oid': '5dd72af19e09317019934a73'},IRAM LIEVANOS VELAZQUEZ,E9C1C827AE1234CCF7AC4D9070BB597C,"{'id': 'E9C1C827AE1234CCF7AC4D9070BB597C', 'le...",{'streetAddress': 'PASEO DE LA ASUNCION NO. 53...,"{'name': 'IRAM LIEVANOS VELAZQUEZ', 'email': '...","[tenderer, supplier]"
4,{'$oid': '5dd72af19e09317019934a76'},SEGUROS SURA SA DE CV,R&S811221KR6,"{'id': 'R&S811221KR6', 'legalName': 'SEGUROS S...",{'streetAddress': 'BLVD ADOLFO LOPEZ MATEOS 24...,"{'name': 'Daniel Ernesto De la Fuente Barra', ...","[tenderer, supplier]"


In [ ]:
# bag = db.read_text('../../mongo_data/contrataciones.json', blocksize='50MB')
# bag = bag.map(json.loads)
# bag = bag.map(lambda row: row['cycle']).frequencies()
# # bag = 
# result = bag.compute()
# result